In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import warnings
import matplotlib.pyplot as plt
import numpy as np
import glob
import xarray as xr
import xbudget
import regionate
import xwmt
import xwmb
import xgcm
import cartopy.crs as ccrs
import CM4Xutils #needed to run pip install nc-time-axis
from regionate import MaskRegions, GriddedRegion

In [5]:
print('xgcm version', xgcm.__version__, '\nregionate version', regionate.__version__, '\nxwmt version', xwmt.__version__, '\nxwmb version', xwmb.__version__)

xgcm version 0.8.2.dev52+g5398098 
regionate version 0.3.3 
xwmt version 0.1.0 
xwmb version 0.3.0


### Request HPC Resources

In [5]:
from dask_jobqueue import SLURMCluster  # setup dask cluster 
from dask.distributed import Client

log_directory="/vortexfs1/home/anthony.meza/scratch/CM4XTransientTracers/WaterMassBudgets/logs"

cluster = SLURMCluster(
    cores=36,
    processes=1,
    memory='190GB',
    walltime='04:00:00',
    queue='compute',
    interface='ib0', 
log_directory = log_directory)
print(cluster.job_script())
cluster.scale(jobs=6)

client = Client(cluster)
client

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /vortexfs1/home/anthony.meza/scratch/CM4XTransientTracers/WaterMassBudgets/logs/dask-worker-%J.err
#SBATCH -o /vortexfs1/home/anthony.meza/scratch/CM4XTransientTracers/WaterMassBudgets/logs/dask-worker-%J.out
#SBATCH -p compute
#SBATCH -n 1
#SBATCH --cpus-per-task=36
#SBATCH --mem=177G
#SBATCH -t 04:00:00

/vortexfs1/home/anthony.meza/miniforge3/envs/cm4x_analysis/bin/python -m distributed.cli.dask_worker tcp://172.16.3.124:43750 --name dummy-name --nthreads 36 --memory-limit 176.95GiB --nanny --death-timeout 60 --interface ib0



Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.16.3.124:43750,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Load in data

In [6]:
from tqdm import tqdm

datadir = lambda x="" : "/vortexfs1/home/anthony.meza/scratch/CM4XTransientTracers/data/model/budgets_sigma2_1p5/" + x
datafiles = glob.glob(datadir("CM4Xp125*"))[20:]
datafiles = sorted(datafiles)


ds = xr.open_mfdataset(
        datafiles[1],
        data_vars="minimal",
        coords="minimal",
        compat="override",
        parallel=True,
        engine="zarr")

ds

<xarray.Dataset> Size: 176GB
Dimensions:                         (exp: 2, time: 60, sigma2_l: 74, yh: 224,
                                     xh: 240, yq: 225, xq: 241, sigma2_i: 75,
                                     time_bounds: 61)
Coordinates: (12/30)
    areacello                       (yh, xh) float64 430kB dask.array<chunksize=(224, 240), meta=np.ndarray>
    deptho                          (yh, xh) float64 430kB dask.array<chunksize=(224, 240), meta=np.ndarray>
    dxCv                            (yq, xh) float64 432kB dask.array<chunksize=(225, 240), meta=np.ndarray>
    dyCu                            (yh, xq) float64 432kB dask.array<chunksize=(224, 241), meta=np.ndarray>
  * exp                             (exp) object 16B 'forced' 'control'
    geolat                          (yh, xh) float64 430kB dask.array<chunksize=(224, 240), meta=np.ndarray>
    ...                              ...
    wet_u                           (yh, xq) float64 432kB dask.array<chunksize=(224, 241), meta=np.ndarray>
    wet_v                           (yq, xh) float64 432kB dask.array<chunksize=(225, 240), meta=np.ndarray>
  * xh                              (xh) int64 2kB 0 1 2 3 4 ... 236 237 238 239
  * xq                              (xq) int64 2kB 0 1 2 3 4 ... 237 238 239 240
  * yh                              (yh) int64 2kB 0 1 2 3 4 ... 220 221 222 223
  * yq                              (yq) int64 2kB 0 1 2 3 4 ... 221 222 223 224
Data variables: (12/54)
    EVAP                            (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    LSNK                            (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    LSRC                            (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    RAIN                            (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    SNOWFL                          (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    S_advection_xy                  (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    ...                              ...
    umo                             (exp, time, sigma2_l, yh, xq) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 241), meta=np.ndarray>
    vert_remap_h_tendency           (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    vmo                             (exp, time, sigma2_l, yq, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 225, 240), meta=np.ndarray>
    vprec                           (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    wfo                             (exp, time, sigma2_l, yh, xh) float64 4GB dask.array<chunksize=(2, 1, 74, 224, 240), meta=np.ndarray>
    zos                             (exp, time, yh, xh) float64 52MB dask.array<chunksize=(2, 1, 224, 240), meta=np.ndarray>
Attributes:
    NCO:                 netCDF Operators version 5.0.1 (Homepage = http://nc...
    associated_files:    areacello: 18550101.D2ocean_static.nc
    description:         The CM4Xp125 experimental design following Griffies ...
    external_variables:  volcello
    filename:            ocean_monthly_z_d2.185501-185912.S_advection_xy.nc
    grid_tile:           N/A
    grid_type:           regular
    history:             Sun Jun 11 03:29:11 2023: ncatted -O -a associated_f...
    model:               CM4Xp125
    provenance:          Diagnostics have been conservatively remapped into m...
    title:               CM4_historical_c192_OM4p125
    version:             v0.3.0

In [14]:
from tqdm import tqdm

datadir = lambda x="" : "/vortexfs1/home/anthony.meza/scratch/CM4XTransientTracers/data/model/budgets_sigma2_1p5/" + x
datafiles = glob.glob(datadir("CM4Xp125*"))[20:]
datafiles = sorted(datafiles)

dss = []

for (t, file) in enumerate(tqdm(datafiles)): 
    ds = xr.open_mfdataset(
        file,
        data_vars="minimal",
        coords="minimal",
        compat="override",
        parallel=True,
        engine="zarr")

    ds_sfc = ds[["fsitherm", "tos", "zos", "sos", "hfsso", "rsdoabsorb", "siconc", "taux", "tauy"]] #select variables of interest 
    ds_sfc = ds_sfc.sel(yh = slice(None, 70))
    ds_sfc = ds_sfc.groupby("time.year").mean("time") #use yearly data 
    dss += [ds_sfc.copy(deep=True)]

100%|██████████| 50/50 [00:28<00:00,  1.75it/s]


In [15]:
dss_ds = xr.concat(dss, dim = "year", combine_attrs="override").compute()

dataset_size_bytes = dss_ds.nbytes
dataset_size_gb = dataset_size_bytes / (1024**3)
print(f"Dataset size: {dataset_size_gb:.2f} GB")

/vortexfs1/home/anthony.meza/miniforge3/envs/cm4x_analysis/lib/python3.12/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 12.66 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Dataset size: 14.47 GB


In [16]:
savedir = "/vortexfs1/home/anthony.meza/scratch/CM4XTransientTracers/data/model/"
dss_ds.to_netcdf(savedir + f"Southern_Ocean_50S_Yearly_Surface_Fluxes_And_Properties.nc")

In [2]:
ds

NameError: name 'ds' is not defined